## 자연어 처리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Oshima
- 부동산 사건 사고가 표기된 info 컬럼 데이터의 주요 키워드 추출  

In [2]:
import pandas as pd

In [5]:
colab_path = '/content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing/yunyoung/data'
vscode_path = './data'
open = pd.read_csv(colab_path + '/open_data_add_latlng.csv', index_col=0)
airbnb = pd.read_csv(colab_path+'/airbnb_data_add_latlng.csv', index_col=0)
oshima_data = pd.read_csv(colab_path+'/oshima_data_add_latlng.csv', index_col=0)
oshima_data.head()

,district,address,info,date,full name,latitude,longitude
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044


In [ ]:
oshima_data['info'].value_counts()[-10:]

二階、階段をあがって二番目の部屋\n生活保護者の孤高死                                                                        1
60代男性が孤高死                                                                                          1
建築完成後入居前、飛び降り自殺                                                                                    1
50代男性が病死                                                                                           1
桐ヶ丘中学校男子生徒飛び降り自殺                                                                                   1
1階 角部屋 南向き1K（洋室6帖、K1帖） 専有面積15.5㎡\n告知事項あり\n生活保護相談（60代まで）                                            1
やよい荘１０１号室 死体発見（死後３週間前後と推定）                                                                         1
賃貸一戸建て\n鉄骨造 2階建（屋上あり） 主要採光面：南向き5DK（ DK8帖、洋室9帖、洋室4帖、和室6畳、和室6畳、和室6畳） 専有面積135.12㎡\n告知事項有\n定期借家（1年）    1
死体発見 ．                                                                                             1
多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅                                                          

### 데이터 살펴보기

In [ ]:
import ast
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

In [ ]:
pip install ipadic==1.0.0a2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 13.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0a2-py3-none-any.whl size=13556740 sha256=d3048b4ebfcc6a83e236b3142eb25f7b56ed09333b8e7d19b3a4d52f3b777f8a
  Stored in directory: /Users/yunyoung/Library/Caches/pip/wheels/44/f9/02/862651969e5626d2fcc219224a366568ca5cce1eb70c2a48b4
Successfully built ipadic
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.font_manager as font_manager

font_list = font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
font_list[:10]

['/System/Library/Fonts/Supplemental/NotoSansMarchen-Regular.ttf',
 '/System/Library/Fonts/Supplemental/Galvji.ttc',
 '/Library/Fonts/Arial Unicode.ttf',
 '/System/Library/Fonts/Supplemental/SignPainter.ttc',
 '/System/Library/Fonts/Supplemental/DevanagariMT.ttc',
 '/System/Library/Fonts/Supplemental/NotoSansImperialAramaic-Regular.ttf',
 '/System/Library/Fonts/Supplemental/NotoSansBhaiksuki-Regular.ttf',
 '/Users/yunyoung/Library/Fonts/NanumBrush.ttf',
 '/System/Library/Fonts/Supplemental/NotoSansSiddham-Regular.ttf',
 '/System/Library/Fonts/Supplemental/Times New Roman Bold.ttf']

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import MeCab

m = MeCab.Tagger()


def extract_nouns(text):
    node = m.parseToNode(text)
    nouns = []
    while node:
        if node.feature.split(',')[0] == '名詞':  # Extract only nouns
            nouns.append(node.surface)
        node = node.next
    return nouns

# Apply the function to each row in the 'info' column
oshima_data['nouns'] = oshima_data['info'].apply(extract_nouns)
oshima_data[:10]

,district,address,info,date,full name,latitude,longitude,nouns
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,"[死体, 発見]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,[自殺]
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,"[2, 階, サウナ, ロイヤル, 死体, 発見]"
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,[自殺]
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,"[首吊り, 自殺]"
5,新宿区,東京都新宿区西新宿七丁目5-25,20階\n心理的瑕疵あり,不明,東京都 新宿区 西新宿 七丁目 5番 25号,35.697594,139.698837,"[20, 階, 心理, 瑕疵, あり]"
6,新宿区,東京都新宿区歌舞伎町二丁目35-6,コインロッカー内に乳児の腐乱死体。,平成30年6月12日,東京都 新宿区 歌舞伎町 二丁目 35番 6号,35.696503,139.702469,"[コイン, ロッカー, 乳児, 腐乱, 死体]"
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,"[902, 号, 自殺, あり]"
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,"[伊勢丹, テナント, 関係, 首吊り, 自殺]"
9,新宿区,東京都新宿区新宿五丁目18-20 ビジネスホテルミラベル,309号室 首吊り自殺,平成20年12月4日,東京都 新宿区 新宿 五丁目 18番 20号,35.693932,139.706482,"[309, 号, 首吊り, 自殺]"


### 일본어 -> 한국어 번역

In [ ]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ja', target='ko')

def translate(x):
    translated_text = translator.translate(x)
    return translated_text

In [ ]:
from tqdm.notebook import tqdm_notebook

tqdm_notebook()

oshima_data['translate'] = ""

for i, row in tqdm_notebook(oshima_data.iterrows(), total=len(oshima_data)):
    oshima_data['translate'][i] = translate(row['info'])

0it [00:00, ?it/s]

  0%|          | 0/4833 [00:00<?, ?it/s]

<ipython-input-7-12f0b8a1dd51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oshima_data['translate'][i] = translate(row['info'])


In [ ]:
oshima_data.to_csv('translate.csv')


### 태깅하기

In [2]:
cd /content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing

/content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing


In [2]:
import pandas as pd

oshima_kr = pd.read_csv('../yunyoung/data/translate.csv', index_col=0)
oshima_kr

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
4844,多摩市,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃,東京都 八王子市 大塚 640番地,35.637493,139.422913,이웃실에서 수수께끼의 죽음. 당시 제가 살았던 307호실에 수수께끼의 검은 그림자가...
4845,多摩市,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃,東京都 八王子市 大塚 769番地,35.638660,139.424469,대표 이사가 연인의 게이와 싸우고 자살 사고
4846,多摩市,東京都多摩市和田2009-1,飛び降り自殺,2018年2月,東京都 多摩市 和田 2009番地,35.639858,139.427063,뛰어 내리는 자살
4847,多摩市,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日,東京都 八王子市 大塚,35.639523,139.418701,1993년경일까. 10호관 근처에서 학생이 뛰어내려 자살을 했다. 몇 층부터는 잊었...


#### 주소 기준 중복값 제거

In [5]:
oshima_kr_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3644 entries, 0 to 4836
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   3644 non-null   object 
 1   address    3644 non-null   object 
 2   info       3644 non-null   object 
 3   date       3644 non-null   object 
 4   full name  3644 non-null   object 
 5   latitude   3644 non-null   float64
 6   longitude  3644 non-null   float64
 7   translate  3644 non-null   object 
dtypes: float64(2), object(6)
memory usage: 256.2+ KB


In [3]:
df_unique = oshima_kr.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택


In [7]:
df_unique['info'].unique()

array(['死体発見', '飛び降り自殺', '2階サウナロイヤル 死体発見', ..., '自殺&死体発見',
       '多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅',
       '多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅'], dtype=object)

In [9]:
# !git clone https://github.com/uoneway/KoBertSum.git


Cloning into 'KoBertSum'...
remote: Enumerating objects: 908, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 908 (delta 108), reused 102 (delta 102), pack-reused 790
Receiving objects: 100% (908/908), 15.97 MiB | 13.02 MiB/s, done.
Resolving deltas: 100% (576/576), done.


### Korean News Summarization Model : https://huggingface.co/gogamza/kobart-summarization

In [15]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00


In [16]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


In [19]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [23]:
df_unique['translate'][0]

'시체 발견'

In [24]:
raw_input_ids = tokenizer.encode(df_unique['translate'][0])
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]


### 오시마 데이터에 적절한 모델이 아님

In [25]:
summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'시체 발견으로 시체 처리장치가 작동되는 시체 발견으로 시체 발견'

In [31]:
cd /content

/content


### KeyBert , kiwipiepy

In [24]:
!pip install keybert
!pip install kiwipiepy


  Using cached keybert-0.7.0-py3-none-any.whl
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Obtaining dependency information for rich>=10.4.0 from https://files.pythonhosted.org/packages/8d/5f/21a93b2ec205f4b79853ff6e838e3c99064d5dbe85ec6b05967506f14af0/rich-13.5.2-py3-none-any.whl.metadata
  Using cached rich-13.5.2-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for markdown-it-py>=2.2.0 from https://files.pythonhosted.org/packages/42/d7/1ec15b46af6af88f19b8e5ffea08fa375d433c998b8a7639e76935c14f1f/markdown_it_py-3.0.0-py3-none-any.whl.metadata
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached torch-2.0.1

### Sample

In [39]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

text="""
김영하가 『살인자의 기억법』 이후 9 년 만에 내놓는 장편소설 『작별인사』는 그리 멀지 않은 미래를 배경으로, 별안간 삶이 송두리째 뒤흔들린 한 소년의 여정을 좇는다.
유명한 IT 기업의 연구원인 아버지와 쾌적하고 평화롭게 살아가던 철이는 어느날 갑자기 수용소로 끌려가 난생처음 날것의 감정으로 가득한 혼돈의 세계에 맞닥뜨리게 되면서 정신적, 신체적 위기에 직면한다.
동시에 자신처럼 사회에서 배제된 자들을 만나 처음으로 생생한 소속감을 느끼고 따뜻한 우정도 싹틔운다. 철이는 그들과 함께 수용소를 탈출하여 집으로 돌아가기 위해 길을 떠나지만 그 여정에는 피할 수 없는 질문이 기다리고 있다.
세계보건기구 WHO가 팬데믹을 선언한 지 2년이 지나서야 작가는 『작별인사』의 개작을 마쳤다. 420매 분량이던 원고는 약 800매로 늘었고, 주제도 완전히 달라졌다.
‘인간을 인간으로 만드는 것은 무엇인가?’, ‘인간과 인간이 아닌 존재들을 가르는 경계는 어디인가’를 묻던 소설은 ‘삶이란 과연 계속될 가치가 있는 것인가?’,
‘세상에 만연한 고통을 어떻게 하면 줄일 수 있을 것인가’, ‘어쩔 수 없이 태어났다면 어떻게 살고 어떻게 죽어야 할 것인가’와 같은 질문을 던지는 이야기로 바뀌었다.
팬데믹이 개작에 영향을 주었을 수도 있고, 원래 『작별인사』의 구상에 담긴 어떤 맹아가 오랜 개작을 거치며 발아했는지도 모른다. 그것에 대해 작가는 이렇게 말하고 있다.
마치 제목이 어떤 마력이 있어서 나로 하여금 자기에게 어울리는 이야기로 다시 쓰도록 한 것 같은 느낌이다. 탈고를 하고 얼마 지나지 않아 원고를 다시 읽어보았다.
이제야 비로소 애초에 내가 쓰려고 했던 어떤 것이 제대로, 남김 없이 다 흘러나왔다는 생각이 들었다. _’작가의 말’에서
전면적인 수정을 통해 2022년의 『작별인사』는 2020년의 『작별인사』를 마치 시놉시스나 초고처럼 보이게 할 정도로 확연하게 달라졌다. 그리고 김영하의 이전 문학 세계와의 연결점들이 분명해졌다.
"""


In [40]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
keywords

[('장편소설', 0.531),
 ('소설은', 0.5247),
 ('위기에', 0.5216),
 ('삶이란', 0.5162),
 ('삶이', 0.5157),
 ('작가는', 0.5111),
 ('정신적', 0.5075),
 ('생생한', 0.5043),
 ('문학', 0.5007),
 ('연구원인', 0.4899)]

In [41]:
kiwi = Kiwi()
kiwi.analyze(text)

[([Token(form='김영하', tag='NNP', start=1, len=3),
   Token(form='가', tag='JKS', start=4, len=1),
   Token(form='『', tag='SSO', start=6, len=1),
   Token(form='살인자', tag='NNG', start=7, len=3),
   Token(form='의', tag='JKG', start=10, len=1),
   Token(form='기억', tag='NNG', start=12, len=2),
   Token(form='법', tag='NNG', start=14, len=1),
   Token(form='』', tag='SSC', start=15, len=1),
   Token(form='이후', tag='NNG', start=17, len=2),
   Token(form='9', tag='SN', start=20, len=1),
   Token(form='년', tag='NNB', start=22, len=1),
   Token(form='만', tag='NNB', start=24, len=1),
   Token(form='에', tag='JKB', start=25, len=1),
   Token(form='내놓', tag='VV', start=27, len=2),
   Token(form='는', tag='ETM', start=29, len=1),
   Token(form='장편', tag='NNG', start=31, len=2),
   Token(form='소설', tag='NNG', start=33, len=2),
   Token(form='『', tag='SSO', start=36, len=1),
   Token(form='작별', tag='NNG', start=37, len=2),
   Token(form='인사', tag='NNG', start=39, len=2),
   Token(form='』', tag='SSC', start

In [26]:
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [43]:
nouns = noun_extractor(text)
nouns

['김영하',
 '살인자',
 '기억',
 '이후',
 '장편',
 '소설',
 '작별',
 '인사',
 '미래',
 '배경',
 '소년',
 '여정',
 '유명',
 'IT',
 '기업',
 '연구원',
 '아버지',
 '평화',
 '철이',
 '수용소',
 '날것',
 '감정',
 '혼돈',
 '세계',
 '정신',
 '신체',
 '위기',
 '직면',
 '동시',
 '자신',
 '사회',
 '배제',
 '처음',
 '소속감',
 '우정',
 '철이',
 '수용소',
 '탈출',
 '여정',
 '질문',
 '세계보건기구',
 'WHO',
 '팬데믹',
 '선언',
 '작가',
 '작별',
 '인사',
 '개작',
 '분량',
 '원고',
 '주제',
 '인간',
 '인간',
 '무엇',
 '인간',
 '인간',
 '존재',
 '경계',
 '어디',
 '소설',
 '계속',
 '가치',
 '세상',
 '만연',
 '고통',
 '질문',
 '이야기',
 '팬데믹',
 '개작',
 '영향',
 '원래',
 '작별',
 '인사',
 '구상',
 '맹아',
 '개작',
 '발아',
 '그것',
 '작가',
 '제목',
 '마력',
 '자기',
 '이야기',
 '느낌',
 '탈고',
 '얼마',
 '원고',
 '애초',
 '생각',
 '작가',
 '전면',
 '수정',
 '작별',
 '인사',
 '작별',
 '인사',
 '시놉시스',
 '초고',
 '정도',
 '김영하',
 '이전',
 '문학',
 '세계',
 '연결']

In [44]:
text = ' '.join(nouns)
text


'김영하 살인자 기억 이후 장편 소설 작별 인사 미래 배경 소년 여정 유명 IT 기업 연구원 아버지 평화 철이 수용소 날것 감정 혼돈 세계 정신 신체 위기 직면 동시 자신 사회 배제 처음 소속감 우정 철이 수용소 탈출 여정 질문 세계보건기구 WHO 팬데믹 선언 작가 작별 인사 개작 분량 원고 주제 인간 인간 무엇 인간 인간 존재 경계 어디 소설 계속 가치 세상 만연 고통 질문 이야기 팬데믹 개작 영향 원래 작별 인사 구상 맹아 개작 발아 그것 작가 제목 마력 자기 이야기 느낌 탈고 얼마 원고 애초 생각 작가 전면 수정 작별 인사 작별 인사 시놉시스 초고 정도 김영하 이전 문학 세계 연결'

In [45]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('원래', 0.6794),
 ('세상', 0.6729),
 ('이전', 0.6568),
 ('어디', 0.6205),
 ('원고', 0.6179),
 ('애초', 0.6105),
 ('김영하', 0.6049),
 ('팬데믹', 0.6007),
 ('전면', 0.6001),
 ('존재', 0.5995),
 ('인간', 0.5976),
 ('자기', 0.5787),
 ('초고', 0.5772),
 ('탈고', 0.5747),
 ('이야기', 0.5726),
 ('정도', 0.5673),
 ('정신', 0.5624),
 ('처음', 0.5576),
 ('세계', 0.5512),
 ('시놉시스', 0.5475)]

### oshimadata - Keybert

In [54]:
print(df_unique['translate'][6])
nouns = noun_extractor(df_unique['translate'][6])
nouns

동전 사물함에 유아의 부란 시체.


['동전', '사물함', '유아', '시체']

In [53]:
text = ' '.join(nouns)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('유아', 0.881), ('시체', 0.5597), ('사물함', 0.4752), ('동전', 0.2498)]

In [57]:
df_unique['keyword'] = df_unique['translate'].map(noun_extractor)
df_unique

<ipython-input-57-8f68586a727d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['keyword'] = df_unique['translate'].map(noun_extractor)


,district,address,info,date,full name,latitude,longitude,translate,keyword
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,"[시체, 발견]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,[자살]
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,"[사우나, 로얄, 시체, 발견]"
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,[자살]
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,[자살]
...,...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음,"[심리, 하자]"
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시...","[심리, 하자, 설명, LIFULL, HOME, S, 에이블, 카미이시, 카미이점]"
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견,"[자살, 시체, 발견]"
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택,"[타마, 뉴타운, 영산, 단지, 호실, 특별, 모집, 주택]"


In [25]:
def get_topN_keyword(keyword):
  text = ' '.join(keyword)
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
  return keywords


In [60]:
df_unique['Top 5 keyword'] = df_unique['keyword'].map(get_topN_keyword)
df_unique.head(10)

<ipython-input-60-9c85fb9f6d77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['Top 5 keyword'] = df_unique['keyword'].map(get_topN_keyword)


,district,address,info,date,full name,latitude,longitude,translate,keyword,Top 5 keyword
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,"[시체, 발견]","[(발견, 0.8794), (시체, 0.8008)]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,[자살],"[(자살, 1.0)]"
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,"[사우나, 로얄, 시체, 발견]","[(사우나, 0.9334), (로얄, 0.7064), (발견, 0.6721), (시..."
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,[자살],"[(자살, 1.0)]"
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,[자살],"[(자살, 1.0)]"
5,新宿区,東京都新宿区西新宿七丁目5-25,20階\n心理的瑕疵あり,不明,東京都 新宿区 西新宿 七丁目 5番 25号,35.697594,139.698837,20층\n심리적 하자 있음,"[심리, 하자]","[(하자, 0.8549), (심리, 0.8209)]"
6,新宿区,東京都新宿区歌舞伎町二丁目35-6,コインロッカー内に乳児の腐乱死体。,平成30年6月12日,東京都 新宿区 歌舞伎町 二丁目 35番 6号,35.696503,139.702469,동전 사물함에 유아의 부란 시체.,"[동전, 사물함, 유아, 시체]","[(유아, 0.881), (시체, 0.5597), (사물함, 0.4752), (동전..."
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,902호실에서 자살 있음.,"[호실, 자살]","[(자살, 0.9446), (호실, 0.5454)]"
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,이세탄 테넌트 관계자의 목 매달아 자살,"[이세탄, 테넌트, 관계자, 자살]","[(자살, 0.7929), (이세탄, 0.6237), (테넌트, 0.5543), (..."
9,新宿区,東京都新宿区新宿五丁目18-20 ビジネスホテルミラベル,309号室 首吊り自殺,平成20年12月4日,東京都 新宿区 新宿 五丁目 18番 20号,35.693932,139.706482,309호실 목 매달아 자살,"[호실, 자살]","[(자살, 0.9446), (호실, 0.5454)]"


### 0828 태깅

In [2]:
import pandas as pd

oshima_kr = pd.read_csv('../yunyoung/data/translate.csv', index_col=0)
oshima_kr

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
4844,多摩市,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃,東京都 八王子市 大塚 640番地,35.637493,139.422913,이웃실에서 수수께끼의 죽음. 당시 제가 살았던 307호실에 수수께끼의 검은 그림자가...
4845,多摩市,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃,東京都 八王子市 大塚 769番地,35.638660,139.424469,대표 이사가 연인의 게이와 싸우고 자살 사고
4846,多摩市,東京都多摩市和田2009-1,飛び降り自殺,2018年2月,東京都 多摩市 和田 2009番地,35.639858,139.427063,뛰어 내리는 자살
4847,多摩市,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日,東京都 八王子市 大塚,35.639523,139.418701,1993년경일까. 10호관 근처에서 학생이 뛰어내려 자살을 했다. 몇 층부터는 잊었...


#### 주소 기준 중복값 제거

In [6]:
oshima_kr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4833 entries, 0 to 4848
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   4833 non-null   object 
 1   address    4833 non-null   object 
 2   info       4833 non-null   object 
 3   date       4833 non-null   object 
 4   full name  4833 non-null   object 
 5   latitude   4833 non-null   float64
 6   longitude  4833 non-null   float64
 7   translate  4833 non-null   object 
dtypes: float64(2), object(6)
memory usage: 339.8+ KB


In [7]:
df_unique = oshima_kr.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택


In [17]:
keyword_mapping = {
    
    '火災': '화재',
    '死体': '시체',
    '遺体' : '시체',
    'ミイラ' : '시체',
    '入居者死亡を発見' : '시체',
    'トイレで心肺停止女性発見' : '시체',
    '自殺': '자살',
    '首吊り' : '자살',
    '飛び降り' : '자살',
    '無理心中' : '자살',
    '心中' : '자살',
    '縊死' : '자살',
    '患者死亡' : '자살',
    '孤高死': '고독사',
    '孤独死' : '고독사',
    'て独' : '고독사',
    '餓死' : '고독사',
    '告知事項' : '부동산고지',
    '心理的瑕疵' : '부동산고지',
    '転落死': '추락사',
    'から転落' : '추락사',
    '転落' : '추락사',
    '刺殺': '타살',
    '絞殺' : '타살',
    '射殺' : '타살',
    '車両事故死' : '타살',
    '傷害致死' : '타살',
    '撲殺' : '타살',
    '殺人' : '타살',
    '殴殺' : '타살',
    '虐待' : '타살',
    '遺棄' : '타살',
    '病死' : '질병사',
    '過労死' : '질병사',
    '自然死' : '질병사',
    '熱中症による死亡' : '질병사',
    '業務上過失致死' : '질병사',
    '患者死亡' : '질병사',
    '倒れていて死亡' : '질병사',
    '事故死' : '사고사',
    '傷害のち死亡' : '사고사',
    'エレベータと壁に挟まれ死亡' : '사고사',
    '中毒' : '사고사',
    '死亡事故' : '사고사',
    '溺死' : '익사',
    '水没' : '익사',
    '変死' : '의문사',
    '不審死' : '의문사', 
    '原因不明の事故' : '의문사',
    '突然死' : '의문사',
    '未解決事件' : '의문사',
    '不自然死' : '의문사',
    '不明' : '의문사',
    '精神的瑕疵' : '심신미약',
    '事故物件' : '주택내사고',
    '特別募集' : '주택내사고',
    '事故住宅' : '주택내사고',
    '患者死亡' : '주택내사고',
    '特別募集住宅' : '기타'}


In [18]:
import numpy as np
df_unique.loc[:, 'crimeType'] = np.nan

for idx in range(df_unique.shape[0]):
    for key, value in keyword_mapping.items():
        try:
            if key in str(df_unique.loc[idx, 'info']):
                df_unique.loc[idx, 'crimeType'] = value
                break
        except KeyError:
                continue



/var/folders/ks/_xxh3ssj6dvg4pzv1_tb_bzw0000gn/T/ipykernel_43508/1201421633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.loc[:, 'crimeType'] = np.nan
/var/folders/ks/_xxh3ssj6dvg4pzv1_tb_bzw0000gn/T/ipykernel_43508/1201421633.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_unique.loc[:, 'crimeType'] = np.nan


In [19]:
df_crime_na = df_unique.loc[df_unique['crimeType'].isna(), ['info', 'translate', 'crimeType']]
df_crime_na

,info,translate,crimeType
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN
25,2012.飛降り,2012. 비강,NaN
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN
49,告知事项あり,고지사 项 있음,NaN
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN
...,...,...,...
3513,瑕疵物件,하자 부동산,NaN
3579,幼児児童を突き落として殺害,유아아동을 찌르고 살해,NaN
3581,幼児を突き落として殺害,유아를 찌르고 살해,NaN
3590,溺殺,익사,NaN


In [20]:
df_unique[df_unique['info']=='火災による死亡']

,district,address,info,date,full name,latitude,longitude,translate,crimeType
45,新宿区,東京都新宿区歌舞伎町二丁目28-11ホテルまつき1階-2階間踊り場,火災による死亡,平成28年1月7日,東京都 新宿区 歌舞伎町 二丁目 28番 11号,35.696590,139.703430,화재로 인한 사망,화재
74,豊島区,東京都豊島区東池袋四丁目18-15フラワーハイツ2階,火災による死亡,平成20年12月20日,東京都 豊島区 東池袋 四丁目 18番 15号,35.726536,139.720749,화재로 인한 사망,화재
122,杉並区,東京都杉並区阿佐谷南一丁目20-22川染荘101,火災による死亡,平成24年3月22日,東京都 杉並区 阿佐谷 南一丁目 20番 22号,35.700211,139.640106,화재로 인한 사망,화재
132,板橋区,東京都荒川区荒川三丁目30-4,火災による死亡,平成21年12月24日,東京都 荒川区 荒川 三丁目 30番 4号,35.734417,139.781876,화재로 인한 사망,화재
150,足立区,東京都千代田区神田神保町一丁目10柏水堂ビル4階,火災による死亡,平成21年7月3日,東京都 千代田区 神田神保町 一丁目 10番地,35.696121,139.758408,화재로 인한 사망,화재
...,...,...,...,...,...,...,...,...,...
3587,調布市,東京都調布市多摩川三丁目36-11パーク多摩川106,火災による死亡,平成26年1月6日,東京都 調布市 多摩川 三丁目 36番,35.646194,139.530014,화재로 인한 사망,화재
3589,杉並区,東京都杉並区下井草二丁目18-19,火災による死亡,平成24年3月11日,東京都 杉並区 下井草 二丁目 18番 19号,35.720215,139.629898,화재로 인한 사망,화재
3598,杉並区,東京都中野区大和町一丁目2-3市瀬荘202,火災による死亡,平成27年5月17日,東京都 中野区 大和町 一丁目 2番 3号,35.709927,139.653458,화재로 인한 사망,화재
3605,杉並区,東京都中野区中野三丁目34-8藤和シティコープ中野501,火災による死亡,平成20年12月1日,東京都 中野区 中野 三丁目 34番 8号,35.703484,139.664795,화재로 인한 사망,화재


### 태깅 안된 목록

In [21]:
crime_na_info = df_unique.loc[df_unique['crimeType'].isna(), 'info']
set(list(crime_na_info))

{'102号室 初老男性 術後の経過が悪く退院後、室内で死亡',
 '103号室 男性。室内で死亡。',
 '10年以上前だけど治療を受けていた暴力団の幹部？が乱入してきた別の暴力団関係者かなにかに殺される事件があった。テレビのニュースでも大きく報道されていた。',
 '10階非常階段から落下して死亡',
 '13階 死後3ヶ月後異臭で発見される',
 '15:00頃 90代男性が浴室で倒れる、家人が発見？。\n救急搬送はしたようだ。',
 '18歳の子がマンションが飛び込みしぼうしました!',
 '1階2号室 感電死',
 '1階レストランにて死亡。',
 '1階店舗\n容認事項あり',
 '2002年8月31日、中国籍の謝依俤被告(31)は東京都品川区の製麺所兼ラーメン店「早川製麺所」宅に侵入。ナイフで経営者の早川勇さん=当時(64)=と妻の容子さん=同(57)=の首や胸などを刺して殺害した上に、現金約4万7000円や貴金属など約7万円相当を奪うなどした。\n現在は更地化されマンションが建っている。',
 '2012.飛降り',
 '2012飛降り',
 '2015年くらいに101号室でおばあちゃんが亡くなった。息子と住んでいたが、息子のいない間に一人で亡くなった',
 '201号室',
 '201号室 国籍不詳、年齢37歳、女性、食道静脈瘤破裂による死亡',
 '201号室 角部屋\n告示事項有',
 '2022年10月26日午前11時半頃\nブランズ千代田富士見（予定）の建設現場で、\n警備員の男性がバックしてきたトラックの下敷きになり死亡した',
 '2022年11月30日迄に炎のマークを消してください。\n消していただけない場合には法的措置を取らせていただきますので、早急にご対応ください。宜しくお願いいたします。\n貸主兼管理会社 野村不動産パートナーズ株式会社 高田',
 '202号室\n容認事項有\n2019年6月リフォーム済\n和室→洋室シャワールーム洗濯機置場新設',
 '202号室で50～60代の男性が死亡。（令和4年3月3日官報）',
 '203号室\n高齢者が衰弱死\n死後1週間ほどで訪ねてきた市の職員が発見',
 '205号室/5月2日に人が頭から倒れた様な大きな物音がし、5月4日に男性の仕事場の人間からの連絡により救急車、警察が入り、鑑

In [22]:
print(len(crime_na_info))

249


In [32]:
crime_na_df = df_unique.loc[df_unique['crimeType'].isna(), ['info' , 'translate', 'crimeType']]
crime_na_df

,info,translate,crimeType
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN
25,2012.飛降り,2012. 비강,NaN
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN
49,告知事项あり,고지사 项 있음,NaN
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN
...,...,...,...
3513,瑕疵物件,하자 부동산,NaN
3579,幼児児童を突き落として殺害,유아아동을 찌르고 살해,NaN
3581,幼児を突き落として殺害,유아를 찌르고 살해,NaN
3590,溺殺,익사,NaN


### KeyBERT

In [27]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel


In [37]:
kiwi = Kiwi()
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [38]:
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [39]:
def get_topN_keyword(keyword):
  text = ' '.join(keyword)
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
  return keywords

In [40]:
crime_na_df['keyword'] = crime_na_df['translate'].map(noun_extractor)
crime_na_df

,info,translate,crimeType,keyword
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN,"[호텔, 파샤, 호실, 남성, 사망, 발견, 관보]"
25,2012.飛降り,2012. 비강,NaN,[비강]
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN,"[부동산, 업자, 고지, 물건, 보고]"
49,告知事项あり,고지사 项 있음,NaN,[지사]
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN,"[휴식, 코너, 출혈, 사망]"
...,...,...,...,...
3513,瑕疵物件,하자 부동산,NaN,"[하자, 부동산]"
3579,幼児児童を突き落として殺害,유아아동을 찌르고 살해,NaN,"[유아, 아동, 살해]"
3581,幼児を突き落として殺害,유아를 찌르고 살해,NaN,"[유아, 살해]"
3590,溺殺,익사,NaN,[익사]


In [41]:
crime_na_df['Top 5 keyword'] = crime_na_df['keyword'].map(get_topN_keyword)
crime_na_df.head(10)

,info,translate,crimeType,keyword,Top 5 keyword
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN,"[호텔, 파샤, 호실, 남성, 사망, 발견, 관보]","[(호텔, 0.797), (사망, 0.6044), (호실, 0.5981), (관보,..."
25,2012.飛降り,2012. 비강,NaN,[비강],"[(비강, 1.0)]"
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN,"[부동산, 업자, 고지, 물건, 보고]","[(보고, 0.7972), (고지, 0.7475), (물건, 0.719), (업자,..."
49,告知事项あり,고지사 项 있음,NaN,[지사],"[(지사, 1.0)]"
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN,"[휴식, 코너, 출혈, 사망]","[(사망, 0.7399), (출혈, 0.631), (코너, 0.624), (휴식, ..."
56,302の住民が死亡,302명의 주민이 사망,NaN,"[주민, 사망]","[(사망, 0.6361), (주민, 0.6315)]"
101,901注意 制服の警官が何人か入っていった,901주의 유니폼 경찰이 몇 명 들어갔다.,NaN,"[유니폼, 경찰]","[(경찰, 0.7596), (유니폼, 0.7339)]"
119,"3階 角部屋 東向きワンルーム（洋室6帖） 専有面積16㎡ 賃料5.4万円 管理費等2,00...",3층 각 방 동쪽 원룸(양실 6첩) 전유 면적 16㎡ 임대료 5.4만엔 관리비 등 ...,NaN,"[동쪽, 원룸, 전유, 면적, 임대료, 관리비, 용인, 사항]","[(관리비, 0.7521), (사항, 0.7363), (전유, 0.669), (임대..."
127,女性が亡くなられていた,여자가 죽었다.,NaN,[여자],"[(여자, 1.0)]"
158,訂正して下さり、ありがとうございます。\n\n東京都目黒区上目黒2-15-4 マスダビル 増...,정정해 주셔서 감사합니다.\n\n도쿄도 메구로 구 카미 메구로 2-15-4 마스다 ...,NaN,"[정정, 감사, 도쿄도, 메구로, 카미, 메구로, 마스다, 빌딩, 마스다, 타카라]","[(메구로, 0.8375), (빌딩, 0.7903), (마스다, 0.7614), (..."


In [54]:
filtered_df = crime_na_df[crime_na_df['keyword']
filtered_df

,info,translate,crimeType,keyword,Top 5 keyword
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN,"[부동산, 업자, 고지, 물건, 보고]","[(보고, 0.7972), (고지, 0.7475), (물건, 0.719), (업자,..."
607,賃貸の募集が出ていたが同建物内の他の部屋と比較して明らかに家賃が安かったので不動産業者に確認...,임대 모집이 나왔지만 같은 건물의 다른 방에 비해 분명히 집세가 싸서 부동산업자에게...,NaN,"[임대, 모집, 건물, 집세, 부동산, 업자, 확인, 사람, 호실]","[(집세, 0.5945), (업자, 0.574), (호실, 0.5669), (사람,..."


In [62]:
filtered_df = crime_na_df[crime_na_df['keyword'].apply(lambda x: '사망' in x)]
filtered_df

,info,translate,crimeType,keyword,Top 5 keyword
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN,"[호텔, 파샤, 호실, 남성, 사망, 발견, 관보]","[(호텔, 0.797), (사망, 0.6044), (호실, 0.5981), (관보,..."
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN,"[휴식, 코너, 출혈, 사망]","[(사망, 0.7399), (출혈, 0.631), (코너, 0.624), (휴식, ..."
56,302の住民が死亡,302명의 주민이 사망,NaN,"[주민, 사망]","[(사망, 0.6361), (주민, 0.6315)]"
185,◆エスター調布B203号室◆\n死亡している70代くらいの女性と50代くらいの男性発見。外傷...,◆에스터조후 B203호실◆\n사망하고 있는 70대 정도의 여성과 50대 정도의 남성...,NaN,"[에스터조후, B, 호실, 사망, 정도, 여성, 정도, 남성, 발견, 외상, 연락,...","[(상태, 0.683), (사망, 0.6781), (연락, 0.651), (정도, ..."
200,202号室で50～60代の男性が死亡。（令和4年3月3日官報）,202호실에서 50~60대 남성이 사망. (영화 4년 3월 3일 관보),NaN,"[호실, 남성, 사망, 영화, 관보]","[(영화, 0.7469), (사망, 0.725), (호실, 0.6479), (관보,..."
...,...,...,...,...,...
3332,独居70代女性が熱中症で床についたまま死亡、その後ひと月以上経過。警官に寄り腐敗状態で搬送された。,"독거 70대 여성이 열사병으로 바닥에 붙은 채 사망, 그 후 한 달 이상 경과. 경...",NaN,"[독거, 여성, 사병, 바닥, 사망, 이상, 경과, 경찰관, 부패, 상태, 반송]","[(경찰관, 0.4354), (부패, 0.3864), (독거, 0.2497), (사..."
3358,平成２７年５月１１日ころから平成２７年５月２０日ころまでの間で室内で死亡 死亡日特定できず,2015년 5월 11일 무렵부터 2015년 5월 20일 무렵까지의 실내에서 사망 사...,NaN,"[무렵, 무렵, 실내, 사망, 사망, 특정]","[(실내, 0.8494), (무렵, 0.792), (특정, 0.7264), (사망,..."
3414,入居者死亡,입주자 사망,NaN,"[입주자, 사망]","[(입주자, 0.8581), (사망, 0.8072)]"
3503,落下したひさしの下敷きになり死亡,떨어지는 히사시의 깔개가되어 사망,NaN,"[히사시, 깔개, 사망]","[(사망, 0.8589), (히사시, 0.8031), (깔개, 0.7027)]"


In [61]:
filtered_df = crime_na_df[crime_na_df['info'].apply(lambda x: '告示事' in x)]
filtered_df

,info,translate,crimeType,keyword,Top 5 keyword
1837,201号室 角部屋\n告示事項有,201호실 코너룸\n고시 사항 있음,NaN,"[호실, 코너, 고시, 사항]","[(사항, 0.859), (호실, 0.7355), (코너, 0.7266), (고시,..."
3216,告示事項あり,고시 사항 있음,NaN,"[고시, 사항]","[(고시, 0.9227), (사항, 0.8097)]"
3364,告示事項有,고시 사항 있음,NaN,"[고시, 사항]","[(고시, 0.9227), (사항, 0.8097)]"
